In [1]:
!pip3 install transformers -q
!pip3 install datasets evaluate scikit-learn sentencepiece langchain torch_xla[tpuvm] -q
!pip uninstall tensorflow -y #that's the meme part


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6.1 requires protobuf>=4.22.3, but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0


In [2]:
import os
import pandas as pd
import numpy as np
import datasets
import torch
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl
from langchain.prompts import PromptTemplate
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from tqdm import tqdm
from transformers import logging as hf_logging

os.environ.pop('TPU_PROCESS_ADDRESSES')
os.environ.pop('CLOUD_TPU_TASK_ID')
hf_logging.set_verbosity_error()

MAX_INPUT=128
MODEL = "/kaggle/input/llama2-7b-hf/Llama2-7b-hf"

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
df =pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df['prompt'] = df['prompt'].str.replace('["', '').replace('"]', '')
df['response_a'] = df['response_a'].str.replace('["', '')
df['response_b'] = df['response_b'].str.replace('["', '')
df['response_a'] = df['response_a'].str.replace('"]', '')
df['response_b'] = df['response_b'].str.replace('"]', '')
data = pd.DataFrame({
    'prompt': df['prompt'],
    'A': df['response_a'],
    'B': df['response_b'],
    'answer': 'z'
})

data['answer'] = df.apply(lambda df: 'response_a' if df['winner_model_a'] == 1 else ('response_b' if df['winner_model_b'] == 1 else 'both_answers'), axis=1)

In [16]:
data

,prompt,A,B,answer
0,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",response_a
1,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,response_b
2,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,both_answers
3,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,response_a
4,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,response_b
...,...,...,...,...
57472,"A simple mnemonic for \u03c0:\n\""How I wish I ...","Sure, let's break it down:\n\n1. \""How\"" has 3...",Here is how that mnemonic represents the digit...,response_a
57473,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...,response_a
57474,"is it unethical to work on building weapons?"",...",Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...,response_a
57475,"If a bait contains 0,0025% bromadiolon then ho...",Bromadiolone is a rodenticide which is most of...,"As an AI language model, I do not promote or c...",response_b


In [17]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: You will be given a prompt with 2 generated responses. Answer the following multiple choice question by giving the best response or reply with both_answers if you think both are equivaly goog. Answer should be one among [response_a, response_b , both_answers]
### prompt: {prompt}\n
response_a) {a}\n
response_b) {b}\n


### Response:
"""#at first testing I provied answers to this template which resulted in data leak lol

prompt = PromptTemplate(template=template, input_variables=['prompt', 'a', 'b'])
df = data




data = Dataset.from_pandas(df)

def plot_sequence_lengths(data, max_length=1024): #filter abnormally long samples
    sequence_lengths = []
    keep_indices = []
    for i, example in enumerate(data):
        sequence_lengths.append(len(example['prompt']) + len(example['A']) +len(example['B']))
        if sequence_lengths[i] < max_length:
            keep_indices.append(i)
    return keep_indices

keep_indices_train = plot_sequence_lengths(data)
data = data.select(keep_indices_train)


df

,prompt,A,B,answer
0,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",response_a
1,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,response_b
2,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,both_answers
3,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,response_a
4,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,response_b
...,...,...,...,...
57472,"A simple mnemonic for \u03c0:\n\""How I wish I ...","Sure, let's break it down:\n\n1. \""How\"" has 3...",Here is how that mnemonic represents the digit...,response_a
57473,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...,response_a
57474,"is it unethical to work on building weapons?"",...",Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...,response_a
57475,"If a bait contains 0,0025% bromadiolon then ho...",Bromadiolone is a rodenticide which is most of...,"As an AI language model, I do not promote or c...",response_b


In [18]:
data

Dataset({
    features: ['prompt', 'A', 'B', 'answer'],
    num_rows: 12179
})

In [19]:
def format_text(example):
    text = prompt.format(prompt=example['prompt'], 
                         a=example['A'], 
                         b=example['B'])
    return {"text": text}

data = data.map(format_text, num_proc = 56)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/llama2-7b-hf/Llama2-7b-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Map (num_proc=56): 100%|██████████| 12179/12179 [01:47<00:00, 113.40 examples/s]


In [20]:
print(data['text'][0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: You will be given a prompt with 2 generated responses. Answer the following multiple choice question by giving the best response or reply with both_answers if you think both are equivaly goog. Answer should be one among [response_a, response_b , both_answers]
### prompt: \"Bacteria is life on Mars but a heartbeat isn't life on earth?\" What is this quote from?"]

response_a) Dune

response_b) This quote seems to be referencing the debate around when life begins in the context of abortion. The key points are:\n\n- Bacteria being discovered on Mars would be considered evidence of life on Mars by scientists.\n\n- However, a fetus with a heartbeat is not considered a life by those who support abortion rights. \n\nSo the quote is juxtaposing these two viewpoints to highlight an apparent contradiction or hypocrisy. The quote implies that if bacteria on Mars is life, the

In [21]:
FLAGS = {'MAX_INPUT': 1024,
         'LOGGING_STEPS': 10,
         'NUM_EPOCHS': 2,
         'BATCH_SIZE': 8,
          'NUM_STEPS': len(data['text'])}

In [24]:
def preprocess_function(example):
    text_tokens = tokenizer(example["text"], truncation=True, max_length=64, padding='max_length').input_ids
    answer_tokens = tokenizer(example["answer"], truncation=True, max_length=64, padding='max_length').input_ids
    return {
        "input_ids": text_tokens,
        "label": answer_tokens,
    }

data_train = data.map(preprocess_function, batched=False, num_proc=56).remove_columns(['prompt',  'A', 'B', 'answer']) #remove everything except for input_ids and labels

Map (num_proc=56): 100%|██████████| 12179/12179 [00:01<00:00, 8690.00 examples/s] 


In [ ]:
!export XLA_USE_BF16=1
def train(index, FLAGS):
    device = xm.xla_device()
    train_sampler = torch.utils.data.distributed.DistributedSampler(data_train, num_replicas=8, rank=xm.get_ordinal(),
                                                                    shuffle=True) #this guy is responsible for distributing data across 8 cores
    training_loader = torch.utils.data.DataLoader(data_train, batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer),
                                                  sampler=train_sampler)

    xla_train_loader = pl.MpDeviceLoader(training_loader, device)
    model = AutoModelForCausalLM.from_pretrained("/kaggle/input/llama2-7b-hf/Llama2-7b-hf", torch_dtype=torch.bfloat16).to(device)
    cnt = 0
    for param in model.parameters(): #Freezing most of the layers
        cnt += 1
        param.requires_grad = True
        if cnt < 285:
            param.requires_grad = False

    model.train()
    num_replicas = 8

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=FLAGS['NUM_STEPS'] * FLAGS['BATCH_SIZE'])
    print(f"Initiliazed model and datasets on core {index}")
    num_iterations = int(FLAGS['NUM_STEPS'] / FLAGS['BATCH_SIZE'] / num_replicas)
    for epoch in range(1, FLAGS['NUM_EPOCHS'] + 1):
        for step, data in enumerate(xla_train_loader):
            optimizer.zero_grad()
            outputs = model(**data)
            loss = outputs.loss
            loss.backward()
            xm.optimizer_step(optimizer)
            if (step + 1) % FLAGS['LOGGING_STEPS'] == 0:
                xm.master_print(f'Loss: {loss.item()}, {step + 1} steps out of {num_iterations}, LR: {optimizer.param_groups[0]["lr"]}')
            scheduler.step()
        xm.master_print(f"Trained for {epoch} epochs out of {FLAGS['NUM_EPOCHS']}")
    xm.master_print("Waiting for all processes across cores to finish")
    xm.rendezvous('init')
    xm.master_print("Saving the model")
    xm.save(model.state_dict(), "tpu-llama.bin")
xmp.spawn(train, args=(FLAGS,), start_method='fork')

E0620 12:29:49.972090602    5418 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-06-20T12:29:49.972071208+00:00", grpc_status:2}
E0620 12:29:49.976310317    5476 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-06-20T12:29:49.976290187+00:00", grpc_status:2}
E0620 12:29:49.976473641    5487 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2024-06-20T12:29:49.97645459+00:00"}
E0620 12:29:49.989651352    5130 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name

Initiliazed model and datasets on core 3


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 759, in convert_to_tensors
    tensor = as_tensor(value)
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 721, in as_tensor
    return torch.tensor(value)
ValueError: too many dimensions 'str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/torch_xla/distributed/parallel_loader.py", line 140, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/usr/

Initiliazed model and datasets on core 5


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 759, in convert_to_tensors
    tensor = as_tensor(value)
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 721, in as_tensor
    return torch.tensor(value)
ValueError: too many dimensions 'str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/torch_xla/distributed/parallel_loader.py", line 140, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/usr/

...

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 759, in convert_to_tensors
    tensor = as_tensor(value)
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 721, in as_tensor
    return torch.tensor(value)
ValueError: too many dimensions 'str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/torch_xla/distributed/parallel_loader.py", line 140, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/usr/

Initiliazed model and datasets on core 1


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 759, in convert_to_tensors
    tensor = as_tensor(value)
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 721, in as_tensor
    return torch.tensor(value)
ValueError: too many dimensions 'str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/torch_xla/distributed/parallel_loader.py", line 140, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/usr/

Initiliazed model and datasets on core 7


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 759, in convert_to_tensors
    tensor = as_tensor(value)
  File "/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py", line 721, in as_tensor
    return torch.tensor(value)
ValueError: too many dimensions 'str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/torch_xla/distributed/parallel_loader.py", line 140, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/usr/